In [1]:
from torch.optim.lr_scheduler import ReduceLROnPlateau, CyclicLR
from load_utils import prepare_data
from train_utils import *
from seq2seq_model import *
import numpy as np
import torch
from tqdm.notebook import tnrange, tqdm_notebook
from Calculate_BLEU import *
import datetime

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"

In [3]:
q, a, pairs, vector = prepare_data('validation', 'glove.42B.300d/glove.42B.300d.txt', small=True)

Reading validation -------
Read 4217 sentence pairs
Counting words
Counted words:
In questions: 11087 words
In answers: 4542 words


In [4]:
# Add EOS at end of each target sentence
pairs = [[line[0], line[1]+' EOS'] for line in pairs ]

In [5]:
matrix_len = q.n_words
weights_matrix = np.zeros((matrix_len, 300))
word_found = 0
for i, word in enumerate(q.word2index):
    try:
        weights_matrix[i] = vector[word]
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(300, ))

In [6]:
model = Seq2Seq(32, q.n_words, a.n_words, 300, 100, weights_matrix, 0.3, 'concat', device, _)

C:\Users\maren\Anaconda3\envs\deeplearning\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [7]:
model.load_state_dict(torch.load('test_model'))

<All keys matched successfully>

In [8]:
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(11087, 300)
    (gru): GRU(300, 100, batch_first=True, dropout=0.3)
  )
  (decoder): Decoder(
    (embedding): Embedding(4542, 300)
    (dropout): Dropout(p=0.3, inplace=True)
    (attention): Attention(
      (attn): Linear(in_features=200, out_features=100, bias=True)
    )
    (gru): GRU(300, 100, batch_first=True, dropout=0.3)
    (concat): Linear(in_features=200, out_features=100, bias=True)
    (out): Linear(in_features=100, out_features=4542, bias=True)
    (tan): Tanh()
  )
  (softmax): Softmax(dim=1)
)

In [9]:
n_records = len(pairs)
indexes = np.array(range(n_records))
encoder_in, decoder_in, enc_length, seq_length, _ = to_batch_sequence(pairs, q, a, 32, 64, indexes, device)
dec_len = decoder_in.size()[1]

In [10]:
input = encoder_in[0]

In [11]:
encoder_in.size()

torch.Size([32, 47])

In [12]:
input = input.unsqueeze(0)

In [13]:
predictions = model.predict(input, enc_length)

In [14]:
pred_list = [a.index2word[idx] for idx in predictions]
answer = ' '.join(pred_list[:-1])
answer

'that s used to . we got that fixed yesterday . i m a fixed too and that i to PAD to PAD to PAD to PAD PAD to PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD'

In [15]:
question_list = [q.index2word[idx] for idx in input.squeeze().tolist()]
question = ' '.join(question_list)
question

"i've counted nine different cockroaches and i accidentally stepped on another one . PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD"